In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_04_fakefriend")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 14:57:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os
directory = os.path.join(os.getcwd(),"data")
filename = "fakefriends.csv"
filepath = os.path.join(directory, filename)
lines = spark.textFile("file:///"+filepath.replace("\\","/"))
lines

file:////home/lab12/src/data/fakefriends.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [3]:
lines.take(10)

['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21',
 '5,Weyoun,59,318',
 '6,Gowron,37,220',
 '7,Will,54,307',
 '8,Jadzia,38,380',
 '9,Hugh,27,181']

In [ ]:
#평균 친구수를 구하기 
#데이터에서 친구 수 (네번째 열)을 추출하고 이를 기반으로 평균을 계산

In [5]:
fields = lines.map(lambda line: line.split(","))

In [6]:
friend_counts = fields.map(lambda x: int(x[3]))

In [9]:
#합계와 개수 계산
total_friends = friend_counts.reduce(lambda x, y: x + y)
total_count = friend_counts.count()

In [12]:
average_friends = int(total_friends / total_count)
average_friends
#사람을 소수점으로 자를 수 없어서 정수 형태로 값을 출력함.. 사사오입은 안됨 ㅇㅁㅇ

248

In [ ]:
#친구가 가장 많은 인싸 녀석을 찾기

In [13]:
max_friends_record = fields.reduce(lambda x, y: x if int(x[3]) > int(y[3]) else y)

In [19]:
max_friends_record

['353', 'Julian', '64', '499']

In [20]:
# 1. 데이터 분리
fields = lines.map(lambda line: line.split(","))

# 2. 최대 친구 수 계산
max_friends = fields.map(lambda x: int(x[3])).max()

# 3. 최대 친구 수를 가진 모든 사람 찾기
most_popular = fields.filter(lambda x: int(x[3]) == max_friends).collect()

# 4. 결과 출력
for record in most_popular:
    print(f"가장 친구가 많은 사람: ID={record[0]}, 이름={record[1]}, 나이={record[2]}, 친구 수={record[3]}")


가장 친구가 많은 사람: ID=106, 이름=Beverly, 나이=18, 친구 수=499
가장 친구가 많은 사람: ID=278, 이름=Data, 나이=44, 친구 수=499
가장 친구가 많은 사람: ID=353, 이름=Julian, 나이=64, 친구 수=499


In [ ]:
#동점 일 경우랑 같이 비교.. 헷갈리네

In [ ]:
#친구가 100명 이상인 사람들 찾아보기

In [23]:
friends_above_100 = fields.filter(lambda x: int(x[3]) >= 100)
friends_above_100.collect()

[['0', 'Will', '33', '385'],
 ['2', 'Hugh', '55', '221'],
 ['3', 'Deanna', '40', '465'],
 ['5', 'Weyoun', '59', '318'],
 ['6', 'Gowron', '37', '220'],
 ['7', 'Will', '54', '307'],
 ['8', 'Jadzia', '38', '380'],
 ['9', 'Hugh', '27', '181'],
 ['10', 'Odo', '53', '191'],
 ['11', 'Ben', '57', '372'],
 ['12', 'Keiko', '54', '253'],
 ['13', 'Jean-Luc', '56', '444'],
 ['16', 'Weyoun', '22', '323'],
 ['18', 'Jean-Luc', '45', '455'],
 ['19', 'Geordi', '60', '246'],
 ['20', 'Odo', '67', '220'],
 ['21', 'Miles', '19', '268'],
 ['23', 'Keiko', '51', '271'],
 ['25', 'Ben', '21', '445'],
 ['26', 'Julian', '22', '100'],
 ['27', 'Leeta', '42', '363'],
 ['28', 'Martok', '49', '476'],
 ['29', 'Nog', '48', '364'],
 ['30', 'Keiko', '50', '175'],
 ['31', 'Miles', '39', '161'],
 ['32', 'Nog', '26', '281'],
 ['33', 'Dukat', '53', '197'],
 ['34', 'Jean-Luc', '43', '249'],
 ['35', 'Beverly', '27', '305'],
 ['39', 'Morn', '31', '192'],
 ['40', 'Odo', '52', '413'],
 ['41', 'Hugh', '67', '167'],
 ['43', 'Guinan',

In [ ]:
#collect 방식은 데이터가 많아지면 출력된 내용이 깔끔하지 않거나 드라이버 메모리에 부담이 될 수 있다.

In [24]:
count_above_100 = friends_above_100.count()
count_above_100

394

In [ ]:
# 30~39세 나이대 사람들의 친구 평균수를 구해보자!

In [25]:
age_30_to_39 = fields.filter(lambda x: 30 <= int(x[2]) <= 39)
#x[2]는 문자열 형태의 나이이므로, 이를 정수(int)로 변환하여 비교합니다. -> 이해 완료!

In [31]:
friend_counts = age_30_to_39.map(lambda x: int(x[3]))
total_friends = friend_counts.reduce(lambda x, y: x + y)
total_count = friend_counts.count()
total_count

97

In [33]:
average_friends = int(total_friends / total_count)
average_friends

235

In [ ]:
#235명?!?!?! 평균이하..

In [34]:
spark.stop()